In [54]:
import os
import sys
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import numpy as np

In [63]:
all_fighter_cluster = pd.read_csv(r"https://github.com/john-adeojo/ufc_styles/blob/main/data/02_intermediate/all_fighter_clusters.csv?raw=true")
fighter_data = pd.read_csv(r"https://github.com/john-adeojo/ufc_styles/blob/main/data/02_intermediate/fighter_data.csv?raw=true")

In [64]:
all_fighter_cluster['specific_cluster'].value_counts()

2_Welterweight           196
2_Lightweight            191
2_Middleweight           146
1_Featherweight          140
2_Bantamweight           121
1_LightHeavyweight       117
1_Heavyweight            108
1_WomenStrawweight        64
0_Lightweight             60
2_Flyweight               56
1_Welterweight            53
0_Middleweight            53
1_WomenFlyweight          51
1_CatchWeight             39
2_Featherweight           35
0_Bantamweight            30
1_OpenWeight              27
0_LightHeavyweight        21
0_WomenBantamweight       19
0_Welterweight            19
0_Flyweight               18
-1_WomenBantamweight      16
0_Heavyweight             14
0_Featherweight           14
-1_WomenFeatherweight     13
1_Bantamweight            13
1_WomenBantamweight       12
1_Lightweight             12
0_CatchWeight             12
0_WomenFlyweight          10
1_Flyweight                8
0_OpenWeight               7
1_Middleweight             7
0_WomenStrawweight         7
Name: specific

In [56]:
analysis_data_base = fighter_data.merge(right=all_fighter_cluster[['cluster','specific_cluster', 'Fighter_dims', 'weight_class']], how='left', left_on=['Fighter_dims', 'weight_class'], right_on=['Fighter_dims', 'weight_class'])
analysis_data_base = analysis_data_base.dropna(subset=['cluster'])
analysis_data_opp = analysis_data_base.copy()
analysis_data_opp['opp_specific_cluster'] = analysis_data_opp['specific_cluster']
analysis_data = analysis_data_base.merge(right=analysis_data_opp[['weight_class', 'Fighter_dims', 'date', 'opp_specific_cluster']], how='inner', left_on=['weight_class', 'Opponent', 'date'], right_on=['weight_class', 'Fighter_dims', 'date'] )
analysis_data_opposing_styles = analysis_data.loc[analysis_data['opp_specific_cluster'] != analysis_data['specific_cluster']].copy()
analysis_data_opposing_styles.head()

,Unnamed: 0,R_fighter,B_fighter,date,Winner,weight_class,avg_KD,avg_opp_KD,avg_SUB_ATT,avg_opp_SUB_ATT,...,Stance_Southpaw,Stance_Switch,Stance_nan,fighter,method,fighter_result,cluster,specific_cluster,Fighter_dims_y,opp_specific_cluster


In [39]:
import pandas as pd

# Assuming your DataFrame is named 'analysis_data_opposing_styles'
# and the columns to combine are 'specific_cluster' and 'opp_specific_cluster'

def create_style_matchup(row):
    # Combine the two columns and sort them alphabetically
    matchup = sorted([row['specific_cluster'], row['opp_specific_cluster']])
    
    # Join them with ' vs. ' and return the result
    return matchup[0] + ' vs. ' + matchup[1]

# Apply the custom function to each row and create the new 'style_matchup' column
analysis_data_opposing_styles['style_matchup'] = analysis_data_opposing_styles.apply(create_style_matchup, axis=1)

# Display the DataFrame
analysis_data_opposing_styles.sort_values(by='weight_class', inplace=True)


In [40]:
analysis_data_opposing_styles['style_matchup'].value_counts()

1_Lightweight vs. 2_Lightweight                558
1_Welterweight vs. 2_Welterweight              460
1_Middleweight vs. 2_Middleweight              424
1_LightHeavyweight vs. 2_LightHeavyweight      228
0_Featherweight vs. 2_Featherweight            202
1_Bantamweight vs. 3_Bantamweight              170
1_Heavyweight vs. 2_Heavyweight                152
0_Flyweight vs. 2_Flyweight                    134
0_Featherweight vs. 1_Featherweight            114
0_Welterweight vs. 2_Welterweight              100
0_Bantamweight vs. 3_Bantamweight               94
0_WomenFlyweight vs. 1_WomenFlyweight           84
0_Lightweight vs. 2_Lightweight                 70
0_WomenStrawweight vs. 1_WomenStrawweight       58
0_Welterweight vs. 1_Welterweight               54
0_Middleweight vs. 2_Middleweight               48
0_WomenBantamweight vs. 1_WomenBantamweight     44
0_Lightweight vs. 1_Lightweight                 42
0_Middleweight vs. 1_Middleweight               30
0_Heavyweight vs. 2_Heavyweight

In [ ]:
from scripts.chisquareanalysis import ChiSquareAnalysis

# Assuming you have a DataFrame named 'data'
analysis = ChiSquareAnalysis(analysis_data)
result = analysis.run_chisquare_analysis('variable_name')  # Replace 'variable_name' with the actual variable name

